In [ ]:
import numpy as np
import csv
import pandas as pd
import math

In [ ]:
dir_path = "../output/"
csv_path = dir_path + "Arm_vorne_90Grad_scaled.csv"
mot_path = dir_path + "OpenSim.mot"

In [ ]:
def unit_vector(vector):
    return vector / np.linalg.norm(vector)

def angle(v1, v2):
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return np.degrees(np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0)))

def distance(v1, v2):
    if len(v1) != len(v2):
        raise ValueError("Die Vektoren müssen die gleiche Länge haben.")
    
    squared_sum = sum((a - b) ** 2 for a, b in zip(v1, v2))
    return math.sqrt(squared_sum)

def get_data(file_name):
    file_path = dir_path + file_name + ".csv"
    with open(file_path, "r") as f:
        first_line = f.readline().strip()
        df = pd.read_csv(file_path, comment="#")
    return first_line, df

In [ ]:
with open(csv_path, "r") as f:
    first_line = f.readline().strip()
    df = pd.read_csv(csv_path, comment="#")


print(df.describe())

In [ ]:
df2 = pd.read_csv(mot_path, delimiter="\t")
time = df2.iloc[:, 0]

print(df2.info())

In [ ]:
acromion = df.loc[:, "r_acromion_X":"r_acromion_Z"]
humerus = df.loc[:, "r_humerus_epicondyle_X":"r_humerus_epicondyle_Z"]
radius = df.loc[:, "r_radius_styloid_X":"r_radius_styloid_Z"]

acromion = acromion.to_numpy()
humerus = humerus.to_numpy()
radius = radius.to_numpy()

angles = pd.DataFrame(columns=["time", "shoulder_csv", "elbow_csv", "shoulder_mot", "elbow_mot"])

for i in range(len(acromion)):
    arm_v = humerus[i] - acromion[i]
    shoulder_elev = angle(arm_v, np.array([0, -1, 0]))
    elbow_angle = angle(radius[i] - humerus[i], arm_v)
    angles.loc[len(angles)] = {"time": time[i], "shoulder_csv": shoulder_elev, "elbow_csv": elbow_angle, "shoulder_mot": df2.iloc[i]["r_shoulder_elev"], "elbow_mot": df2.iloc[i]["r_elbow_flex"]}
    #print(str(time[i]) + "\t" + str(shoulder_elev) + "\t" + str(elbow_angle) + "\t")

In [ ]:
angles.describe()

In [ ]:
# angles.plot(x=0, y=[1, 2, 3, 4], label=["Schulter_csv", "Ellenbogen_csv", "Schulter_mot", "Ellenbogen_mot"])
angles.plot(x=0, y=[1, 3], label=["Schulter_csv", "Schulter_mot"])
angles.plot(x=0, y=[2, 4], label=["Ellenbogen_csv", "Ellenbogen_mot"])

In [ ]:
angles.describe()

### Export für fake .mot Datei

In [ ]:
with open("output/FAKEMOT.mot", mode="w", newline='') as file:
    writer = csv.writer(file, delimiter='\t')  # Tab als Trenner
    for index, row in angles.iterrows():
        writer.writerow([row["time"], row["shoulder_csv"], row["elbow_csv"]])

#### Längen

In [ ]:
lengths = pd.DataFrame(columns=["frame", "lower_arm", "upper_arm"], dtype=float)
for i in range(len(acromion)):
    lengths.loc[len(lengths)] = {"frame": i, "lower_arm": distance(humerus[i], radius[i]), "upper_arm": distance(acromion[i], humerus[i])}